# Downloading pH glider data from the IOOS Glider Data Assembly Center

_Written by Lori Garzio, June 14, 2023_

[Rutgers Center for Ocean Observing Leadership](https://rucool.marine.rutgers.edu/) (RUCOOL)

This tutorial shows you how to manually download NetCDF files from the [IOOS Glider DAC](https://gliders.ioos.us/). We have to download the delayed-mode datasets because real-time datasets do not have calculated pH included in the files. pH is calculated in the post-processing and QA/QC process, and those delayed-mode files are submitted to the DAC for public access.

In this example, we will be downloading delayed-mode data from the [spring 2021 deployment of ru30](https://gliders.ioos.us/erddap/tabledap/ru30-20210503T1929-delayed.html). You can browse this ERDDAP data page to get information about the deployment, make some graphs, and download the dataset in a variety of formats.

First, navigate to the main IOOS ERDDAP landing page https://gliders.ioos.us/erddap/index.html and enter the dataset you want to download in the search bar.

<img src="../imgs/erddap01.png" width=1000 height=1000 />

Find the dataset you want (in this case the delayed-mode dataset) and click on "data".

<img src="../imgs/erddap02.png" width=1000 height=1000 />

This will bring you to the data access form. In this form, select the variables you want to include in your downloaded file and double-check the optional constraints. A time range is usually auto-selected, so if you want the entire dataset then make sure to clear those constraints for time.

<img src="../imgs/erddap03.png" width=1000 height=1000 />

Scroll down to the bottom of the form, select ".nc - Download a flat, table-like NetCDF-3 binary file with COARDS/CF/ACDD metadata" as the File type, and hit Submit. It might take a while for the download to complete, depending on the size of your file.

<img src="../imgs/erddap04.png" width=1000 height=1000 />